<a href="https://colab.research.google.com/github/vahid-nejad/OCR-Correction/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=4baa2c633d9a85f0d5c7fba2d3ed7b8fc3a434cc2a5338634435d630e863826d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.4 MB/s eta 0:00:00


In [5]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00


In [7]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/ds/read_res.csv") # Path to your csv file
print(df)

                       image  \
0         {011C011C1},{594}:   
1        {011C011C90},{821}:   
2        {011C011C90},{851}:   
3        {011C011C90},{978}:   
4        {011C011C90},{972}:   
...                      ...   
4479   {111TB111TB18},{306}:   
4480   {111TB111TB18},{628}:   
4481   {111TB111TB18},{873}:   
4482   {111TB111TB18},{955}:   
4483  {111TB111TB18},{1434}:   

                                            groundtruth  \
0     \nخدمات مهندسی، ساخت و تولید، اجرا، تامین، ساخ...   
1                                     \nکوکو سبزی\n\t\t   
2                                \nکوکو سیب زمینی\n\t\t   
3                                         \nفلافل\n\t\t   
4                              \nلازانیای سبزیجات\n\t\t   
...                                                 ...   
4479  \nسپس، با خارج نمودن آن از روی تختۀ چوبی، با م...   
4480  \nفعالیت: یک حیوان (اعم از چارپا و یا پرنده) ر...   
4481  \nفکر کنید: دست/u200cهای شخصیت سه/u200cبعدی صح...   
4482  \nهمان/u200cگونه که

In [8]:
df= df.dropna(subset=['easyocr'])

In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [10]:
train_percentage = 0.7
validation_percentage = 0.15
test_percentage = 0.15

temp = dataset.train_test_split(train_size=train_percentage)
temp2 = temp["test"].train_test_split(train_size=validation_percentage / (validation_percentage + test_percentage))

In [11]:
from datasets import DatasetDict

ds= DatasetDict()

In [12]:
ds["train"]=temp["train"]
ds["validation"] = temp2["train"]
ds["test"] = temp2["test"]


In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'groundtruth', 'tesseract', 'easyocr', 'paddle', '__index_level_0__'],
        num_rows: 2680
    })
    validation: Dataset({
        features: ['image', 'groundtruth', 'tesseract', 'easyocr', 'paddle', '__index_level_0__'],
        num_rows: 574
    })
    test: Dataset({
        features: ['image', 'groundtruth', 'tesseract', 'easyocr', 'paddle', '__index_level_0__'],
        num_rows: 575
    })
})

In [14]:
from transformers import AutoTokenizer
checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [15]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) #from transformers import AutoModelForSeq2SeqLM


In [23]:
import torch

In [ ]:


model=torch.load("./models/mt5_plain_full") # Path to the saved model that you have saved in the fine tune phase
print("model loaded")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('google/mt5-base')





In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
test_text_inputs = []
test_targets = []


for i in range(len(ds["test"])):
    test_text_inputs.append(ds["test"][i]["easyocr"])
    test_targets.append(ds["test"][i]["groundtruth"])

In [20]:
test_data = []
for i in range(len(test_text_inputs)):
    test_data.append({'text':test_text_inputs[i],'summary':test_targets[i]})




In [25]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True,tokenizer=tokenizer)
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [26]:
def evaluate(model, tokenizer, data):
    total_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}

    for example in data:
        text = example['text']
        summary = example['summary']

        # Generate the summary using the model
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=128, early_stopping=True)
        generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Calculate the ROUGE scores
        scores = scorer.score(summary, generated_summary)
        for metric, score in scores.items():
            total_scores[metric] += score.fmeasure

    # Calculate the average ROUGE scores
    num_examples = len(data)
    avg_scores = {metric: score / num_examples for metric, score in total_scores.items()}
    return avg_scores






In [ ]:
test_scores = evaluate(model, tokenizer, test_data)
print(f"ROUGE-1: {test_scores['rouge1']:.4f}")
print(f"ROUGE-2: {test_scores['rouge2']:.4f}")
print(f"ROUGE-L: {test_scores['rougeL']:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
